In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc

warnings.filterwarnings("ignore")

# Tabular Playground Series - Dec 2021 

The tabular series on kaggle are meant to help novices in data science field to get acquainted with kaggle competitions.

For this challenge, we will be predicting a full year worth of sales for three items at two stores located in three different countries. This dataset is completely fictional, but contains many effects you see in real-world data, e.g., weekend and holiday effect, seasonality, etc. The dataset is small enough to allow you to try numerous different modeling approaches. 

# Exploratory Data Analysis

This is not finished yet

# Train dataset

Lets first explore the train dataset and subsequently the test dataset.

In [ ]:
train_df = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2022/train.csv", sep=',')

In [ ]:
train_df.head()

In [ ]:
train_df.tail()

In [ ]:
print('Dataset shape: ', train_df.shape)

In [ ]:
train_df.info()

In [ ]:
train_df.dtypes.value_counts()

In [ ]:
train_df.describe().T

In [ ]:
print("There are", train_df.isna().any().sum(), "missing values")

# Date variable exploratory data analysis

In [ ]:
train_df['date']

In [ ]:
# How many days are valid datas e.g not weekends
dates = pd.to_datetime(train_df['date']).apply(lambda x: x.date()).unique()
valid_days = np.count_nonzero(np.is_busday(dates.astype('datetime64')))
print(valid_days, "valid days out of", len(dates))

In [ ]:
# All years of data
years = set(map(lambda date: date.year, dates))
print("Years present in the data:", sorted(years))

In [ ]:
months_each_year = {}
for year in years:
    months_each_year[year] = set([date.month for date in dates if date.year == year])

print("Year and Months of data for each year:")
months_each_year

In [ ]:
days_each_month_per_year = {}
for year in years:
    for month in months_each_year[year]:
        days_each_month_per_year[str(year) + '-' + str(month)] = len([date.day for date in dates if date.year == year and date.month == month])

print("Days of available data for each month each year:")
days_each_month_per_year

# Explanatory variables Distribution (Categorical Variables)

In [ ]:
print("Unique values in every categorical variable\n")
print("country:", train_df['country'].unique())
print()
print("store:", train_df['store'].unique())
print()
print("product:", train_df['product'].unique()) 	 	

<h3> Total Sales made by Country </h3>

In [ ]:
train_df.drop(['row_id'], axis=1).groupby(["country"]).sum()

<h3>Min and Max sales by country</h3>

In [ ]:
train_df.drop(['row_id'], axis=1).groupby(["country"]).min()

In [ ]:
train_df.drop(['row_id'], axis=1).groupby(["country"]).max()

<h3> Mean Sales made by Country </h3>

In [ ]:
train_df.drop(['row_id'], axis=1).groupby(["country"]).mean()

<h3>Total Sales made by country on each store</h3>

In [ ]:
train_df.drop(['row_id'], axis=1).groupby(["country", "store"]).sum()

<h3>Min and Max sales made by country on each store</h3>

In [ ]:
train_df.drop(['row_id'], axis=1).groupby(["country", "store"]).min()

In [ ]:
train_df.drop(['row_id'], axis=1).groupby(["country", "store"]).max()

<h3>Mean Sales made by country on each store</h3>

In [ ]:
train_df.drop(['row_id'], axis=1).groupby(["country", "store"]).mean()

<h3>Total Sales made by country on each store for each product</h3>

In [ ]:
train_df.drop(['row_id'], axis=1).groupby(["country", "store", "product"]).sum()

<h3>Min and Max Sales made by country on each store for each product</h3>

In [ ]:
train_df.drop(['row_id'], axis=1).groupby(["country", "store", "product"]).min()

In [ ]:
train_df.drop(['row_id'], axis=1).groupby(["country", "store", "product"]).max()

<h3>Mean Sales made by country on each store for each product</h3>

In [ ]:
train_df.drop(['row_id'], axis=1).groupby(["country", "store", "product"]).mean()

<h3>The most sold product</h3>

In [ ]:
train_df.drop(['row_id'], axis=1).groupby(["product"]).max()

<h3>The lesser sold product</h3>

In [ ]:
train_df.drop(['row_id'], axis=1).groupby(["product"]).min()

<h3>Count plot of every categorical variable</h3>

The number of items sold is almost the same for every country, every product and every store. Every categorical variable is perfectly balanced.

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 5))
ax = ax.flatten()
columns = train_df.select_dtypes(include='object').drop('date', axis=1).columns

for i, column in enumerate(columns):
    sns.countplot(x=train_df[column], ax=ax[i])

plt.tight_layout()
fig.show()

<h3>Box plot, violin plot and swarmplot of every categorical variable against the target variable (num_sold)</h3>

From the boxplots below we can observe the following:

1. Norway sold a bigger number of items than Finland and Sweden
2. The store Kaggle Rama Sold much more items than the Kaggle Mart;
3. Kaggle Hat has is the product most sold, followed by Kaggle Mug, and Kaggle Sticker the less sold product.

<h4>country  num_sold boxplot and violinplot</h4>

In [ ]:
categorical_vars = ['country', 'store', 'product']

plt.figure(figsize=(10, 5))
sns.boxplot(data = train_df, x='country', y='num_sold')

In [ ]:
plt.figure(figsize=(10, 5))
sns.violinplot(data = train_df, x='country', y='num_sold')

<h4>store  num_sold boxplot and violinplot</h4>

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(data = train_df, x='store', y='num_sold')

In [ ]:
plt.figure(figsize=(10, 5))
sns.violinplot(data = train_df, x='store', y='num_sold')

<h4>product  num_sold boxplot and violinplot</h4>

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(data = train_df, x='product', y='num_sold')

In [ ]:
plt.figure(figsize=(10, 5))
sns.violinplot(data = train_df, x='product', y='num_sold')

In [ ]:
sns.countplot(data = train_df, x = 'country')

train_dfget Variable Distribution

In [ ]:
# Target Distribution
plt.figure(figsize=(12, 7))
sns.kdeplot(data=train_df, x='num_sold', palette = ["#2a9d8f"], shade=True)

<h3>Full Series</h3>

In [ ]:
df = pd.DataFrame({'num_sold' : train_df['num_sold'].values}, index=pd.to_datetime(train_df['date'].values))
fig, ax = plt.subplots(1, 1, figsize=(15, 7))
ax.grid(which='both')
ax.legend(['Target variable (num_sold)'], loc="upper left")
ax.set_ylabel('Number of goods sold', labelpad=20, fontsize=15)
df.plot(ax=ax)
plt.show()

# Yearly series


In [ ]:
df = pd.DataFrame({'num_sold' : train_df['num_sold'].values}, index=pd.to_datetime(train_df['date'].values))
years = df.index.year.unique()

for year in years:
    df_year = df[df.index.year == year]
    fig, ax = plt.subplots(1, 1, figsize=(15, 7))
    ax.grid(which='both')
    ax.legend(['Target variable (num_sold)'], loc="upper left")
    ax.set_ylabel('Number of goods sold', labelpad=20, fontsize=15)
    df_year.plot(ax=ax)
    plt.show()

# Yearly num_sold by country

In [ ]:
df = pd.DataFrame({'num_sold' : train_df['num_sold'].values,
                   'country'  : train_df['country'].values},
                  index=pd.to_datetime(train_df['date'].values))

years = df.index.year.unique()
countries = train_df.country.unique()

for country in countries:
    print(f"\n--- {country} ---\n")
    for year in years:
        df_year = df[(df.index.year == year) & (df.country == country)]
        fig, ax = plt.subplots(1, 1, figsize=(15, 7))
        fig.subplots_adjust(hspace=0.30)
        ax.grid(which='both')
        ax.legend(['Target variable (num_sold)'], loc="upper left")
        ax.set_ylabel('Number of goods sold', labelpad=20, fontsize=15)
        df_year.plot(ax=ax, title=country)
        plt.show()

# Yearly total num_sold by month for each country

In [ ]:
df = pd.DataFrame({'num_sold' : train_df['num_sold'].values,
                   'country'  : train_df['country'].values},
                  
                  index=pd.to_datetime(train_df['date'].values))

df['month'] = df.index.month
years = df.index.year.unique()
countries = train_df.country.unique()

for country in countries:
    print(f"\n--- {country} ---\n")
    for year in years:
        df_year = df[(df.index.year == year) & (df.country == country)]
        df_year = df_year.groupby("month").sum()        
        fig, ax = plt.subplots(1, 1, figsize=(15, 7))
        
        # fig.subplots_adjust(hspace=0.30)
        ax.grid(which='both')
        ax.legend(['Target variable (num_sold)'], loc="upper left")
        ax.set_ylabel('Total number of goods sold monthly', labelpad=20, fontsize=15)
        df_year.plot(ax=ax, title=country + " - " + str(year))
        plt.show()

# Test dataset

For the test dataset we will only check for missing values and if all classes are present in the data. The information gathered below shows that
the test data is similar to the train data.

In [ ]:
test_df = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2022/test.csv", sep=',')

In [ ]:
test_df.head()

In [ ]:
test_df.tail()

In [ ]:
print('Dataset shape: ', test_df.shape)

In [ ]:
test_df.info()

In [ ]:
test_df.dtypes.value_counts()

In [ ]:
print("There are", test_df.isna().any().sum(), "missing values")

In [ ]:
print("Unique values in every categorical variable\n")
print("country:", test_df['country'].unique())
print()
print("store:", test_df['store'].unique())
print()
print("product:", test_df['product'].unique()) 	 	

In [ ]:
test_df['date']

In [ ]:
# How many days are valid datas e.g not weekends
dates = pd.to_datetime(test_df['date']).apply(lambda x: x.date()).unique()
valid_days = np.count_nonzero(np.is_busday(dates.astype('datetime64')))
print(valid_days, "valid days out of", len(dates))

In [ ]:
# All years of data
years = set(map(lambda date: date.year, dates))
print("Years present in the data:", sorted(years))

In [ ]:
months_each_year = {}
for year in years:
    months_each_year[year] = set([date.month for date in dates if date.year == year])

print("Year and Months of data for each year:")
months_each_year

In [ ]:
days_each_month_per_year = {}
for year in years:
    for month in months_each_year[year]:
        days_each_month_per_year[str(year) + '-' + str(month)] = len([date.day for date in dates if date.year == year and date.month == month])

print("Days of available data for each month each year:")
days_each_month_per_year

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 5))
ax = ax.flatten()
columns = test_df.select_dtypes(include='object').drop('date', axis=1).columns

for i, column in enumerate(columns):
    sns.countplot(x=test_df[column], ax=ax[i])

plt.tight_layout()
fig.show()